# Conversion of HHL Algorithm to QCFD Model

#### JOB POSTING ID: **#54**
#### EXPERT NAME/ID: **Samer Rahmeh [sam@samrahmeh.com]** *(samgr55)*
#### Client: **DYNEX Dev**
#### Platform: **DYNEX Marketplace**

------------------------------------------------------------------------------------------------------------------------------------

## Loading dependencies

In [1]:
from qiskit.algorithms import HHL, NumPyLinearSolver
from qiskit import Aer, execute
from qiskit.quantum_info import state_fidelity
import numpy as np
from QCFD import QCFD
import dynex
from dimod import SimulatedAnnealingSampler, BinaryQuadraticModel

## Use-case (Example)
This example has been taken from the paper *An Introduction to Algorithms in Quantum Computation of Fluid Dynamics* 

In [2]:
# Use-case (taken from the paper)
matrix = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                   [0,4,-1,0,0,-2,0,0,0,0,0,0,0,0,0,0],
                   [0,-1,4,0,0,0,-2,0,0,0,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
                   [0,-2,0,0,0,4,-1,0,0,-1,0,0,0,0,0,0],
                   [0,0,-2,0,0,-1,4,0,0,0,-1,0,0,0,0,0],
                   [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
                   [0,0,0,0,0,-1,0,0,0,4,-1,0,0,-2,0,0],
                   [0,0,0,0,0,0,-1,0,0,-1,4,0,0,0,-2,0],
                   [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,0,0,0,-2,0,0,0,4,-1,0],
                   [0,0,0,0,0,0,0,0,0,0,-2,0,0,-1,4,0],
                   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
                   ])
vector = np.array([200,200, 0,0, 200,200, 0,0, 200,200, 0,0, 200,200, 0,0])

**Convert Quantum State into Vector State**: The function `Cir2Vec` takes a quantum state from an HHL solution and utilizes Qiskit's `statevector_simulator` backend to execute the quantum circuit. It returns the resulting state vector, effectively converting the quantum state into a classical vector representation for further analysis or comparison.


In [3]:
# Convert Quantum State into Vector State
def Cir2Vec(HHL_QSol):
    backend = Aer.get_backend('statevector_simulator')
    job = execute(HHL_QSol, backend)
    return job.result().get_statevector()

**Benchmarking with Fidelity**: The `Fidelity` function computes the fidelity between a quantum solution (`QSol`) and a classical solution (`CSol`). It first normalizes both solutions and then utilizes Qiskit's `state_fidelity` function to calculate the fidelity. This metric provides a measure of the similarity between the two solutions, where a fidelity close to 1 indicates a high degree of similarity.

In [4]:
# Benchmarking with Fidelity
def Fidelity(QSol, CSol):
    QSol_Norm = QSol / np.linalg.norm(QSol)
    CSol_Norm = CSol / np.linalg.norm(CSol)
    fidelity = state_fidelity(QSol_Norm, CSol_Norm)
    print("fidelity %f" % fidelity)

**Quantum and Classical Solution Comparison**: This section first calculates the quantum solution using the Harrow-Hassidim-Lloyd (HHL) algorithm and then extracts the state vector of the quantum solution using the `Cir2Vec` function. It prints both the quantum state and its state vector representation. Next, it calculates the classical solution to the same linear system using Qiskit's `NumPyLinearSolver` for comparison. Both solutions are rounded and printed for easy comparison. This allows for a direct comparison of quantum and classical approaches in solving linear systems and benchmarking their results.

In [5]:
# Quantum Solution
hhl = HHL()
hhlRes = hhl.solve(matrix, vector)
QSol = hhlRes.state
print("Quantum Solution State: ")
print(QSol)
QSol_SV = np.round(Cir2Vec(QSol), 5)
print("Quantum Solution StateVector: ")
print(QSol_SV)

# Linear Solution
NPLinear = NumPyLinearSolver()
classicRes = NPLinear.solve(matrix, vector)
CSol = classicRes.state
CSol_SV = np.round(CSol, 5)
print("Classical solution: ")
print(CSol_SV)

Quantum Solution State: 
      ┌────────────┐┌──────┐        ┌─────────┐
q0_0: ┤0           ├┤5     ├────────┤5        ├
      │            ││      │        │         │
q0_1: ┤1           ├┤6     ├────────┤6        ├
      │  circuit-7 ││      │        │         │
q0_2: ┤2           ├┤7     ├────────┤7        ├
      │            ││      │        │         │
q0_3: ┤3           ├┤8     ├────────┤8        ├
      └────────────┘│      │┌──────┐│         │
q1_0: ──────────────┤0 QPE ├┤4     ├┤0 QPE_dg ├
                    │      ││      ││         │
q1_1: ──────────────┤1     ├┤3     ├┤1        ├
                    │      ││      ││         │
q1_2: ──────────────┤2     ├┤2     ├┤2        ├
                    │      ││  1/x ││         │
q1_3: ──────────────┤3     ├┤1     ├┤3        ├
                    │      ││      ││         │
q1_4: ──────────────┤4     ├┤0     ├┤4        ├
                    └──────┘│      │└─────────┘
q2_0: ──────────────────────┤5     ├───────────
               

----------------------------------------------------------------------------------------

## Loading QCFD

**Initializing QCFD Class**: Here, we instantiate the `QCFD` class, creating an object `qcfd` to be used to access and execute the methods defined in the `QCFD` class, such as converting linear systems to QUBO format `Lin2QUBO` and solving them using various computational methods `compute` and more.

In [6]:
qcfd = QCFD()

**Executing QCFD Computation Locally**: This line calls the `compute` method of the `qcfd` object, passing the matrix and vector as parameters along with the specification `compute='local'`. This instructs the QCFD class to solve the converted QUBO problem using the local computational method `CPU` (`Simulated Annealing Sampler`)

**[NOTE]:** `SAsol` is the solution obtained from the local computation decoded back to vectorized array `DecodeSol`

In [16]:
SAsol = qcfd.compute(matrix, vector, compute='local')

[199. 159. 127.   0. 199. 221. 185.   0. 199. 255. 227.   0. 199. 216.
 192.   0.]


### Comparing Classical and Quantum Solutions with Fidelity
This section utilizes the `Fidelity` function to compare the classical solution obtained from `NumPyLinearSolver()` with the quantum solution obtained from the QCFD model using Simulated Annealing `compute='local'` as the QUBO solver. By invoking `Fidelity(SAsol, CSol_SV)`, it calculates and prints the fidelity between the two solutions, providing a measure of similarity. A higher fidelity indicates a closer match between the classical and quantum solutions, serving as a benchmark for the effectiveness and accuracy of the quantum computational approach used in the QCFD model.

In [17]:
### Using Qiskit state_fidelity to compare 
### The Classical Solution with Quantum Solution (Simulated Annealing QUBO)
### [NumPyLinearSolver()] vs. [QCFD()] 
Fidelity(SAsol,CSol_SV) 

fidelity 0.978069


Make sure my marketplace account is ready for active on DYNEX

In [9]:
dynex.test()

[DYNEX] TEST: dimod BQM construction...
[DYNEX] PASSED
[DYNEX] TEST: Dynex Sampler object...
[DYNEX] PASSED
[DYNEX] TEST: submitting sample file...
[DYNEX] PASSED
[DYNEX] TEST: retrieving samples...
[DYNEX] PASSED
[DYNEX] TEST RESULT: ALL TESTS PASSED


**Executing QCFD Computation on DYNEX**: This line calls the `compute` method of the `qcfd` object, passing the matrix and vector as parameters along with the specification `compute='dynex'`. This instructs the QCFD class to solve the converted QUBO problem using DYNEX Network `Blockchain` (`DYNEX Neuromorphic Computing`)

In [12]:

Dsol = qcfd.compute(matrix, vector, compute='dynex')

╭────────────┬─────────────┬───────────┬────────────────┬─────────┬─────────┬────────────────╮
│   DYNEXJOB │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CHIPS │   STEPS │   GROUND STATE │
├────────────┼─────────────┼───────────┼────────────────┼─────────┼─────────┼────────────────┤
│        996 │        0.98 │      7.94 │              3 │    1216 │     200 │    14504524.00 │
╰────────────┴─────────────┴───────────┴────────────────┴─────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬─────────┬───────┬─────────────┬──────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CHIPS │   LOC │      ENERGY │      RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼─────────┼───────┼─────────────┼──────────────┼─────────────────────────────┼───────────┼──────────┤
│ a5e8..72b9 │ 2.3.5.OZM.127.L │     480 │     0 │        0.00 │ 4.746618721s │ 2023-12-25T12:40:06.433377Z │ 0 (0.00%) │  STO

### Comparing Classical and DYNEX Solutions with Fidelity
This section utilizes the `Fidelity` function to compare the classical solution obtained from `NumPyLinearSolver()` with the Dynex solution obtained from the QCFD model using Neuromorphic Computing `compute='dynex'` as the QUBO solver. By invoking `Fidelity(Dsol, CSol_SV)`, it calculates and prints the fidelity between the two solutions, providing a measure of similarity. A higher fidelity indicates a closer match between the classical and Dynex solutions, serving as a benchmark for the effectiveness and accuracy of the Dynex computational approach used in the QCFD model.

In [13]:
### Using Qiskit state_fidelity to compare 
### The Classical Solution with Neuromorphic Solution (DYNEX Platform)
### [NumPyLinearSolver] vs. [QCFD()] 
Fidelity(Dsol,CSol_SV) 

fidelity 0.985251
